# Demo Endpoints Testing

This notebook provides comprehensive testing for all demo endpoints in the FastAPI CRUD application.

## Endpoints Covered:

### Products:
- GET `/demo/products/` - List all products
- POST `/demo/products/` - Create new product
- GET `/demo/products/{product_id}` - Get product by ID
- PUT `/demo/products/{product_id}` - Update product
- DELETE `/demo/products/{product_id}` - Delete product
- GET `/demo/products/search` - Search products

### Orders:
- GET `/demo/orders/` - List all orders
- POST `/demo/orders/` - Create new order
- GET `/demo/orders/{order_id}` - Get order by ID
- PUT `/demo/orders/{order_id}` - Update order
- DELETE `/demo/orders/{order_id}` - Delete order
- GET `/demo/orders/user/{user_id}` - Get orders by user

In [ ]:
# Import required libraries
import requests
import json
from datetime import datetime
import time
from pprint import pprint
import uuid
from decimal import Decimal

# Configuration
BASE_URL = "http://localhost:8000"
API_V1_STR = "/api/v1"
AUTH_URL = f"{BASE_URL}{API_V1_STR}/auth"
DEMO_URL = f"{BASE_URL}{API_V1_STR}/demo"

# Admin credentials for authentication
admin_user = {"email": "admin@example.com", "password": "secret"}

# Global variables
admin_token = None
created_product_id = None
created_order_id = None
test_user_id = None

print(f"Testing demo endpoints at: {DEMO_URL}")

## Setup: Authentication

Authenticate as admin to access demo endpoints.

In [ ]:
# Setup authentication
def setup_authentication():
    global admin_token, test_user_id

    # Get admin token
    login_data = {"username": admin_user["email"], "password": admin_user["password"]}

    response = requests.post(f"{AUTH_URL}/login/access-token", data=login_data)

    if response.status_code == 200:
        data = response.json()
        admin_token = data.get("access_token")
        print(f"✅ Admin authenticated successfully")
        print(f"Token: {admin_token[:50]}...")

        # Get current user info to get user ID
        headers = {"Authorization": f"Bearer {admin_token}"}
        user_response = requests.get(
            f"{BASE_URL}{API_V1_STR}/users/me", headers=headers
        )
        if user_response.status_code == 200:
            user_data = user_response.json()
            test_user_id = user_data.get("id")
            print(f"Admin user ID: {test_user_id}")

        return True
    else:
        print(f"❌ Authentication failed: {response.text}")
        return False


# Run setup
auth_success = setup_authentication()
if auth_success:
    print("\n🎉 Setup completed successfully!")
else:
    print("\n❌ Setup failed. Please check server and credentials.")

## 1. Product Management Testing

Test all product-related CRUD operations.

In [ ]:
# Test product operations
def test_product_operations():
    global created_product_id

    if not admin_token:
        print("❌ Admin token not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("🛍️ Testing Product Operations")
    print("=" * 40)

    # 1. Create a new product
    print("\n1. Creating a new product:")
    new_product = {
        "name": f"Test Product {int(time.time())}",
        "description": "This is a test product for API testing",
        "price": 29.99,
        "category": "Electronics",
        "stock_quantity": 100,
        "is_active": True,
    }

    response = requests.post(f"{DEMO_URL}/products", headers=headers, json=new_product)
    print(f"Status: {response.status_code}")

    if response.status_code == 201:
        print("✅ Product created successfully")
        product_data = response.json()
        created_product_id = product_data.get("id")
        print(f"Product ID: {created_product_id}")
        print(f"Product Name: {product_data.get('name')}")
        print(f"Price: ${product_data.get('price')}")
    else:
        print(f"❌ Product creation failed: {response.text}")
        return

    # 2. Get all products
    print("\n2. Getting all products:")
    response = requests.get(f"{DEMO_URL}/products", headers=headers)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Products retrieved successfully")
        products = response.json()
        print(f"Total products: {len(products)}")
        for i, product in enumerate(products[:3]):  # Show first 3
            print(f"  {i+1}. {product.get('name')} - ${product.get('price')}")
    else:
        print(f"❌ Failed to get products: {response.text}")

    # 3. Get product by ID
    print("\n3. Getting product by ID:")
    response = requests.get(
        f"{DEMO_URL}/products/{created_product_id}", headers=headers
    )
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Product retrieved by ID")
        product = response.json()
        print(f"Name: {product.get('name')}")
        print(f"Description: {product.get('description')}")
        print(f"Stock: {product.get('stock_quantity')}")
    else:
        print(f"❌ Failed to get product by ID: {response.text}")

    # 4. Update product
    print("\n4. Updating product:")
    update_data = {
        "name": f"Updated Test Product {int(time.time())}",
        "price": 39.99,
        "stock_quantity": 150,
    }

    response = requests.put(
        f"{DEMO_URL}/products/{created_product_id}", headers=headers, json=update_data
    )
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Product updated successfully")
        updated_product = response.json()
        print(f"New name: {updated_product.get('name')}")
        print(f"New price: ${updated_product.get('price')}")
        print(f"New stock: {updated_product.get('stock_quantity')}")
    else:
        print(f"❌ Failed to update product: {response.text}")

    # 5. Search products
    print("\n5. Searching products:")
    search_params = {"query": "Test"}
    response = requests.get(
        f"{DEMO_URL}/products/search", headers=headers, params=search_params
    )
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Product search successful")
        search_results = response.json()
        print(f"Search results: {len(search_results)} products found")
        for product in search_results[:2]:  # Show first 2
            print(f"  - {product.get('name')}")
    else:
        print(f"❌ Product search failed: {response.text}")


test_product_operations()

## 2. Order Management Testing

Test all order-related CRUD operations.

In [ ]:
# Test order operations
def test_order_operations():
    global created_order_id

    if not admin_token or not created_product_id or not test_user_id:
        print("❌ Required data not available (token, product_id, or user_id)")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("📦 Testing Order Operations")
    print("=" * 40)

    # 1. Create a new order
    print("\n1. Creating a new order:")
    new_order = {
        "user_id": test_user_id,
        "items": [
            {"product_id": created_product_id, "quantity": 2, "unit_price": 39.99}
        ],
        "status": "pending",
        "shipping_address": "123 Test Street, Test City, TC 12345",
        "notes": "Test order created via API testing",
    }

    response = requests.post(f"{DEMO_URL}/orders", headers=headers, json=new_order)
    print(f"Status: {response.status_code}")

    if response.status_code == 201:
        print("✅ Order created successfully")
        order_data = response.json()
        created_order_id = order_data.get("id")
        print(f"Order ID: {created_order_id}")
        print(f"Total Amount: ${order_data.get('total_amount')}")
        print(f"Status: {order_data.get('status')}")
        print(f"Items: {len(order_data.get('items', []))}")
    else:
        print(f"❌ Order creation failed: {response.text}")
        return

    # 2. Get all orders
    print("\n2. Getting all orders:")
    response = requests.get(f"{DEMO_URL}/orders", headers=headers)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Orders retrieved successfully")
        orders = response.json()
        print(f"Total orders: {len(orders)}")
        for i, order in enumerate(orders[:3]):  # Show first 3
            print(
                f"  {i+1}. Order {order.get('id')} - ${order.get('total_amount')} ({order.get('status')})"
            )
    else:
        print(f"❌ Failed to get orders: {response.text}")

    # 3. Get order by ID
    print("\n3. Getting order by ID:")
    response = requests.get(f"{DEMO_URL}/orders/{created_order_id}", headers=headers)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Order retrieved by ID")
        order = response.json()
        print(f"Order ID: {order.get('id')}")
        print(f"User ID: {order.get('user_id')}")
        print(f"Total: ${order.get('total_amount')}")
        print(f"Status: {order.get('status')}")
        print(f"Shipping: {order.get('shipping_address')[:50]}...")
    else:
        print(f"❌ Failed to get order by ID: {response.text}")

    # 4. Update order
    print("\n4. Updating order:")
    update_data = {
        "status": "processing",
        "notes": "Order updated via API testing - now processing",
    }

    response = requests.put(
        f"{DEMO_URL}/orders/{created_order_id}", headers=headers, json=update_data
    )
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Order updated successfully")
        updated_order = response.json()
        print(f"New status: {updated_order.get('status')}")
        print(f"Updated notes: {updated_order.get('notes')}")
    else:
        print(f"❌ Failed to update order: {response.text}")

    # 5. Get orders by user
    print("\n5. Getting orders by user:")
    response = requests.get(f"{DEMO_URL}/orders/user/{test_user_id}", headers=headers)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ User orders retrieved successfully")
        user_orders = response.json()
        print(f"Orders for user {test_user_id}: {len(user_orders)}")
        for order in user_orders:
            print(
                f"  - Order {order.get('id')}: ${order.get('total_amount')} ({order.get('status')})"
            )
    else:
        print(f"❌ Failed to get user orders: {response.text}")


test_order_operations()

## 3. Advanced Testing - Pagination and Filtering

Test pagination, filtering, and advanced query parameters.

In [ ]:
# Test advanced features
def test_advanced_features():
    if not admin_token:
        print("❌ Admin token not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("🔍 Testing Advanced Features")
    print("=" * 40)

    # 1. Product pagination
    print("\n1. Product pagination:")
    params = {"skip": 0, "limit": 2}
    response = requests.get(f"{DEMO_URL}/products", headers=headers, params=params)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Product pagination working")
        products = response.json()
        print(f"Products (limit 2): {len(products)}")

    # 2. Product filtering by category
    print("\n2. Product filtering by category:")
    params = {"category": "Electronics"}
    response = requests.get(f"{DEMO_URL}/products", headers=headers, params=params)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Product filtering working")
        products = response.json()
        print(f"Electronics products: {len(products)}")

    # 3. Product filtering by price range
    print("\n3. Product filtering by price range:")
    params = {"min_price": 20, "max_price": 50}
    response = requests.get(f"{DEMO_URL}/products", headers=headers, params=params)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Price range filtering working")
        products = response.json()
        print(f"Products in $20-$50 range: {len(products)}")
        for product in products[:2]:
            print(f"  - {product.get('name')}: ${product.get('price')}")

    # 4. Order filtering by status
    print("\n4. Order filtering by status:")
    params = {"status": "processing"}
    response = requests.get(f"{DEMO_URL}/orders", headers=headers, params=params)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Order status filtering working")
        orders = response.json()
        print(f"Processing orders: {len(orders)}")

    # 5. Order sorting
    print("\n5. Order sorting by creation date:")
    params = {"sort_by": "created_at", "sort_order": "desc"}
    response = requests.get(f"{DEMO_URL}/orders", headers=headers, params=params)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        print("✅ Order sorting working")
        orders = response.json()
        print(f"Orders (newest first): {len(orders)}")
        for i, order in enumerate(orders[:2]):
            created_at = order.get("created_at", "N/A")
            print(f"  {i+1}. Order {order.get('id')} - {created_at}")


test_advanced_features()

## 4. Error Cases and Validation Testing

Test various error scenarios and input validation.

In [ ]:
# Test error cases
def test_error_cases():
    if not admin_token:
        print("❌ Admin token not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("⚠️ Testing Error Cases")
    print("=" * 40)

    # 1. Access without authentication
    print("\n1. Access endpoint without authentication:")
    response = requests.get(f"{DEMO_URL}/products")
    print(f"Status: {response.status_code} (expected: 401)")
    if response.status_code == 401:
        print("✅ Correctly requires authentication")

    # 2. Get non-existent product
    print("\n2. Get non-existent product:")
    fake_uuid = str(uuid.uuid4())
    response = requests.get(f"{DEMO_URL}/products/{fake_uuid}", headers=headers)
    print(f"Status: {response.status_code} (expected: 404)")
    if response.status_code == 404:
        print("✅ Correctly returns 404 for non-existent product")

    # 3. Create product with invalid data
    print("\n3. Create product with invalid data:")
    invalid_product = {
        "name": "",  # Empty name
        "price": -10,  # Negative price
        "stock_quantity": "invalid",  # Invalid type
    }
    response = requests.post(
        f"{DEMO_URL}/products", headers=headers, json=invalid_product
    )
    print(f"Status: {response.status_code} (expected: 422)")
    if response.status_code == 422:
        print("✅ Correctly validates product data")
        error_details = response.json()
        print(f"Validation errors: {len(error_details.get('detail', []))}")

    # 4. Create order with non-existent product
    print("\n4. Create order with non-existent product:")
    invalid_order = {
        "user_id": test_user_id,
        "items": [
            {
                "product_id": str(uuid.uuid4()),  # Non-existent product
                "quantity": 1,
                "unit_price": 10.00,
            }
        ],
        "status": "pending",
    }
    response = requests.post(f"{DEMO_URL}/orders", headers=headers, json=invalid_order)
    print(f"Status: {response.status_code} (expected: 400 or 404)")
    if response.status_code in [400, 404]:
        print("✅ Correctly handles non-existent product in order")

    # 5. Update with invalid UUID
    print("\n5. Update with invalid UUID format:")
    invalid_uuid = "not-a-valid-uuid"
    update_data = {"name": "Updated Name"}
    response = requests.put(
        f"{DEMO_URL}/products/{invalid_uuid}", headers=headers, json=update_data
    )
    print(f"Status: {response.status_code} (expected: 422)")
    if response.status_code == 422:
        print("✅ Correctly validates UUID format")

    # 6. Search with invalid parameters
    print("\n6. Search with very long query:")
    long_query = "x" * 1000  # Very long search query
    params = {"query": long_query}
    response = requests.get(
        f"{DEMO_URL}/products/search", headers=headers, params=params
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Handles long search queries gracefully")
    elif response.status_code == 422:
        print("✅ Correctly validates search query length")


test_error_cases()

## 5. Performance and Load Testing

Test performance with multiple requests and measure response times.

In [ ]:
# Test performance
def test_performance():
    if not admin_token:
        print("❌ Admin token not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("⚡ Testing Performance")
    print("=" * 40)

    # 1. Multiple product requests
    print("\n1. Multiple product list requests:")
    start_time = time.time()
    request_count = 10

    for i in range(request_count):
        response = requests.get(f"{DEMO_URL}/products", headers=headers)
        if response.status_code != 200:
            print(f"❌ Request {i+1} failed with status {response.status_code}")

    end_time = time.time()
    total_time = end_time - start_time
    avg_time = total_time / request_count

    print(f"✅ {request_count} requests completed")
    print(f"Total time: {total_time:.2f} seconds")
    print(f"Average time per request: {avg_time:.3f} seconds")
    print(f"Requests per second: {request_count/total_time:.2f}")

    # 2. Concurrent search requests
    print("\n2. Search performance test:")
    search_queries = ["test", "product", "electronics", "demo", "api"]
    start_time = time.time()

    for query in search_queries:
        params = {"query": query}
        response = requests.get(
            f"{DEMO_URL}/products/search", headers=headers, params=params
        )
        if response.status_code == 200:
            results = response.json()
            print(f"  Query '{query}': {len(results)} results")

    search_time = time.time() - start_time
    print(f"Search tests completed in {search_time:.2f} seconds")

    # 3. Order retrieval performance
    print("\n3. Order retrieval performance:")
    start_time = time.time()

    # Test different pagination sizes
    page_sizes = [5, 10, 20]
    for size in page_sizes:
        params = {"limit": size}
        response = requests.get(f"{DEMO_URL}/orders", headers=headers, params=params)
        if response.status_code == 200:
            orders = response.json()
            print(f"  Limit {size}: {len(orders)} orders retrieved")

    pagination_time = time.time() - start_time
    print(f"Pagination tests completed in {pagination_time:.2f} seconds")


test_performance()

## 6. Data Integrity and Business Logic Testing

Test business rules and data consistency.

In [ ]:
# Test business logic
def test_business_logic():
    if not admin_token or not created_product_id:
        print("❌ Required data not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("🧠 Testing Business Logic")
    print("=" * 40)

    # 1. Test order total calculation
    print("\n1. Order total calculation:")
    order_items = [
        {"product_id": created_product_id, "quantity": 3, "unit_price": 39.99}
    ]

    expected_total = 3 * 39.99

    new_order = {"user_id": test_user_id, "items": order_items, "status": "pending"}

    response = requests.post(f"{DEMO_URL}/orders", headers=headers, json=new_order)

    if response.status_code == 201:
        order_data = response.json()
        actual_total = float(order_data.get("total_amount", 0))
        print(f"Expected total: ${expected_total:.2f}")
        print(f"Actual total: ${actual_total:.2f}")

        if (
            abs(actual_total - expected_total) < 0.01
        ):  # Allow for floating point precision
            print("✅ Order total calculation is correct")
        else:
            print("❌ Order total calculation is incorrect")
    else:
        print(f"❌ Failed to create order: {response.text}")

    # 2. Test product stock tracking
    print("\n2. Product stock consistency:")

    # Get current stock
    response = requests.get(
        f"{DEMO_URL}/products/{created_product_id}", headers=headers
    )
    if response.status_code == 200:
        product = response.json()
        current_stock = product.get("stock_quantity", 0)
        print(f"Current stock: {current_stock}")

        # Update stock
        new_stock = current_stock + 50
        update_data = {"stock_quantity": new_stock}

        update_response = requests.put(
            f"{DEMO_URL}/products/{created_product_id}",
            headers=headers,
            json=update_data,
        )

        if update_response.status_code == 200:
            updated_product = update_response.json()
            updated_stock = updated_product.get("stock_quantity", 0)
            print(f"Updated stock: {updated_stock}")

            if updated_stock == new_stock:
                print("✅ Stock update is consistent")
            else:
                print("❌ Stock update is inconsistent")

    # 3. Test order status transitions
    print("\n3. Order status transitions:")
    if created_order_id:
        valid_statuses = ["pending", "processing", "shipped", "delivered", "cancelled"]

        for status in ["shipped", "delivered"]:
            update_data = {"status": status}
            response = requests.put(
                f"{DEMO_URL}/orders/{created_order_id}",
                headers=headers,
                json=update_data,
            )

            if response.status_code == 200:
                updated_order = response.json()
                new_status = updated_order.get("status")
                print(f"  Status updated to: {new_status}")

                if new_status == status:
                    print(f"  ✅ Status transition to '{status}' successful")
                else:
                    print(f"  ❌ Status transition to '{status}' failed")
            else:
                print(f"  ❌ Failed to update status to '{status}': {response.text}")


test_business_logic()

## 7. Cleanup and Delete Operations

Test delete operations and cleanup test data.

In [ ]:
# Test cleanup operations
def test_cleanup_operations():
    if not admin_token:
        print("❌ Admin token not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    print("🗑️ Testing Cleanup Operations")
    print("=" * 40)

    # 1. Delete order
    if created_order_id:
        print("\n1. Deleting test order:")
        response = requests.delete(
            f"{DEMO_URL}/orders/{created_order_id}", headers=headers
        )
        print(f"Status: {response.status_code}")

        if response.status_code == 200:
            print("✅ Order deleted successfully")

            # Verify deletion
            verify_response = requests.get(
                f"{DEMO_URL}/orders/{created_order_id}", headers=headers
            )
            if verify_response.status_code == 404:
                print("✅ Order deletion verified")
            else:
                print("❌ Order still exists after deletion")
        else:
            print(f"❌ Failed to delete order: {response.text}")

    # 2. Delete product
    if created_product_id:
        print("\n2. Deleting test product:")
        response = requests.delete(
            f"{DEMO_URL}/products/{created_product_id}", headers=headers
        )
        print(f"Status: {response.status_code}")

        if response.status_code == 200:
            print("✅ Product deleted successfully")

            # Verify deletion
            verify_response = requests.get(
                f"{DEMO_URL}/products/{created_product_id}", headers=headers
            )
            if verify_response.status_code == 404:
                print("✅ Product deletion verified")
            else:
                print("❌ Product still exists after deletion")
        else:
            print(f"❌ Failed to delete product: {response.text}")

    # 3. Test deleting non-existent item
    print("\n3. Deleting non-existent product:")
    fake_uuid = str(uuid.uuid4())
    response = requests.delete(f"{DEMO_URL}/products/{fake_uuid}", headers=headers)
    print(f"Status: {response.status_code} (expected: 404)")

    if response.status_code == 404:
        print("✅ Correctly handles deletion of non-existent item")


# Uncomment to run cleanup (this will delete test data)
# test_cleanup_operations()

## 8. Complete Demo Test Suite

Run comprehensive tests for all demo endpoints.

In [ ]:
# Complete demo test suite
def run_complete_demo_test_suite():
    print("🧪 Running Complete Demo Endpoints Test Suite")
    print("=" * 60)
    
    results = {
        "timestamp": datetime.now().isoformat(),
        "tests": {},
        "summary": {}
    }
    
    # Setup authentication
    if not setup_authentication():
        print("❌ Authentication failed. Cannot proceed with tests.")
        return results
    
    headers = {"Authorization": f"Bearer {admin_token}"}
    
    # Test cases for products
    product_tests = [
        ("get_products", lambda: requests.get(f"{DEMO_URL}/products", headers=headers)),
        ("create_product", lambda: requests.post(f"{DEMO_URL}/products", headers=headers, json={
            "name": f"Suite Test Product {int(time.time())}",
            "description": "Test product",
            "price": 19.99,
            "category": "Test",
            "stock_quantity": 50
        })),
        ("search_products", lambda: requests.get(f"{DEMO_URL}/products/search", headers=headers, params={"query": "test"})),
    ]
    
    # Test cases for orders
    order_tests = [
        ("get_orders", lambda: requests.get(f"{DEMO_URL}/orders", headers=headers)),
    ]
    
    all_tests = product_tests + order_tests
    
    print(f"\n🧪 Running {len(all_tests)} test cases...")
    
    for test_name, test_func in all_tests:
        print(f"\n  Testing {test_name}...")
        try:
            response = test_func()
            success = 200 <= response.status_code < 300
            
            results["tests"][test_name] = {
                "status_code": response.status_code,
                "success": success,
                "response_time": response.elapsed.total_seconds(),
                "response_size": len(response.text)
            }
            
            status = "✅ PASS" if success else "❌ FAIL"
            print(f"    Result: {status} (Status: {response.status_code}, Time: {response.elapsed.total_seconds():.3f}s)")
            
            # Store created IDs for subsequent tests
            if test_name == "create_product" and success:
                product_data = response.json()
                global created_product_id
                created_product_id = product_data.get("id")
                
        except Exception as e:
            results["tests"][test_name] = {
                "status_code": None,
                "success": False,
                "error": str(e)
            }
            print(f"    Result: ❌ ERROR - {str(e)}")
    
    # Additional tests if product was created successfully
    if created_product_id:
        additional_tests = [
            ("get_product_by_id", lambda: requests.get(f"{DEMO_URL}/products/{created_product_id}", headers=headers)),
            ("update_product", lambda: requests.put(f"{DEMO_URL}/products/{created_product_id}", headers=headers, json={
                "name": "Updated Test Product",
                "price": 29.99
            })),
        ]
        
        print(f"\n🔄 Running additional tests with created product...")
        
        for test_name, test_func in additional_tests:
            print(f"\n  Testing {test_name}...")
            try:
                response = test_func()
                success = 200 <= response.status_code < 300
                
                results["tests"][test_name] = {
                    "status_code": response.status_code,
                    "success": success,
                    "response_time": response.elapsed.total_seconds(),
                    "response_size": len(response.text)
                }
                
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"    Result: {status} (Status: {response.status_code})")
                
            except Exception as e:
                results["tests"][test_name] = {
                    "status_code": None,
                    "success": False,
                    "error": str(e)
                }
                print(f"    Result: ❌ ERROR - {str(e)}")
    
    # Calculate summary
    total_tests = len(results["tests"])
    passed_tests = sum(1 for test in results["tests"].values() if test["success"])
    failed_tests = total_tests - passed_tests
    
    avg_response_time = sum(
        test.get("response_time", 0) for test in results["tests"].values() 
        if test.get("response_time") is not None
    ) / max(1, len([t for t in results["tests"].values() if t.get("response_time") is not None]))
    
    results["summary"] = {
        "total": total_tests,
        "passed": passed_tests,
        "failed": failed_tests,
        "success_rate": (passed_tests / total_tests * 100) if total_tests > 0 else 0,
        "avg_response_time": avg_response_time
    }
    
    # Print summary
    print("\n" + "=" * 60)
    print("📊 DEMO ENDPOINTS TEST SUMMARY")
    print("=" * 60)
    print(f"Total Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {failed_tests}")
    print(f"Success Rate: {results['summary']['success_rate']:.1f}%")
    print(f"Average Response Time: {avg_response_time:.3f}s")
    
    if failed_tests > 0:
        print("\n❌ Failed Tests:")
        for test_name, test_result in results["tests"].items():
            if not test_result["success"]:
                error_info = test_result.get('error', f'Status {test_result.get('status_code')}')
                print(f"  - {test_name}: {error_info}")
    
    return results

# Run the complete test suite
demo_test_results = run_complete_demo_test_suite()